In [1]:
!wget https://raw.githubusercontent.com/YookoTora/hse21_hw2/main/gms2.lst

--2022-02-03 18:20:43--  https://raw.githubusercontent.com/YookoTora/hse21_hw2/main/gms2.lst
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 221337 (216K) [text/plain]
Сохранение в: «gms2.lst.5»

gms2.lst.5          100%[===================>] 216,15K  --.-KB/s    за 0,09s   

2022-02-03 18:20:44 (2,31 MB/s) - «gms2.lst.5» сохранён [221337/221337]



In [2]:
!wget https://raw.githubusercontent.com/YookoTora/hse21_hw2/main/scaffolds.hits_from_MIL_1.txt

--2022-02-03 18:20:44--  https://raw.githubusercontent.com/YookoTora/hse21_hw2/main/scaffolds.hits_from_MIL_1.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 873076 (853K) [text/plain]
Сохранение в: «scaffolds.hits_from_MIL_1.txt.2»

scaffolds.hits_from 100%[===================>] 852,61K  3,65MB/s    за 0,2s    

2022-02-03 18:20:45 (3,65 MB/s) - «scaffolds.hits_from_MIL_1.txt.2» сохранён [873076/873076]



In [3]:
!wget https://raw.githubusercontent.com/YookoTora/hse21_hw2/main/proteins.fasta

--2022-02-03 18:20:45--  https://raw.githubusercontent.com/YookoTora/hse21_hw2/main/proteins.fasta
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1402796 (1,3M) [text/plain]
Сохранение в: «proteins.fasta.3»

proteins.fasta.3    100%[===================>]   1,34M   662KB/s    за 2,1s    

2022-02-03 18:20:48 (662 KB/s) - «proteins.fasta.3» сохранён [1402796/1402796]



In [4]:
!wget https://raw.githubusercontent.com/YookoTora/hse21_hw1/main/Poil_gapClosed.fa

--2022-02-03 18:20:48--  https://raw.githubusercontent.com/YookoTora/hse21_hw1/main/Poil_gapClosed.fa
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 3963861 (3,8M) [text/plain]
Сохранение в: «Poil_gapClosed.fa.3»

Poil_gapClosed.fa.3 100%[===================>]   3,78M  3,15MB/s    за 1,2s    

2022-02-03 18:20:50 (3,15 MB/s) - «Poil_gapClosed.fa.3» сохранён [3963861/3963861]



In [5]:
!wget https://raw.githubusercontent.com/YookoTora/hse21_hw2/main/scaffolds.hits_from_SwissProt.txt

--2022-02-03 18:20:51--  https://raw.githubusercontent.com/YookoTora/hse21_hw2/main/scaffolds.hits_from_SwissProt.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 65267 (64K) [text/plain]
Сохранение в: «scaffolds.hits_from_SwissProt.txt.2»

scaffolds.hits_from 100%[===================>]  63,74K  --.-KB/s    за 0,07s   

2022-02-03 18:20:51 (859 KB/s) - «scaffolds.hits_from_SwissProt.txt.2» сохранён [65267/65267]



In [6]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.



In [7]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [8]:
!pip install biopython

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [9]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt
import pandas as pd

In [10]:
scaffolds = dict()
for record in SeqIO.parse("Poil_gapClosed.fa", "fasta"):
    record.annotations['molecule_type'] = 'DNA'
    record.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()
    record.annotations['data_file_division'] = 'BCT'
    scaffolds[record.id] = record

In [11]:
genes = dict()
for gene in SeqIO.parse("proteins.fasta", "fasta"):
    desc = gene.description.split(' ')
    scaffold = desc[1]
    start, end = int(desc[2]), int(desc[3])
    strand = 1 if desc[4] == '+' else -1

    feat = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
    feat.qualifiers['locus_tag'] = [desc[0]]
    feat.qualifiers['translation'] = [gene.seq]
    scaffolds[scaffold].features.append(feat)

    genes[desc[0]] = feat
    

In [12]:
print(feat.qualifiers)

OrderedDict([('locus_tag', ['3614']), ('translation', [Seq('GLPLYRMETIFKRMGIHLPRNTLANWMIKSSECLQPLYNLLNDQLLESGYLHMD...DDY')])])


In [13]:
mil1genes = dict()
for feat in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
    if 'protein_id' not in feat.qualifiers:
        continue
    if 'product' not in feat.qualifiers:
        continue
  
    mil1genes[feat.qualifiers['protein_id'][0]] = feat.qualifiers['product'][0]

In [14]:
names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mil1hits = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)
mil1hits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
1,1,lcl|HF680312.1_prot_CCU72283.1_1834,97.980,99,2,0,1,99,1,99,2.290000e-67,194.0
2,1,lcl|HF680312.1_prot_CCU71933.1_1484,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
3,1,lcl|HF680312.1_prot_CCU71592.1_1143,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
4,1,lcl|HF680312.1_prot_CCU71865.1_1416,96.774,62,2,0,1,62,1,62,2.150000e-42,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10552,3614,lcl|HF680312.1_prot_CCU73517.1_3068,96.552,116,4,0,1,116,216,331,4.090000e-80,241.0
10553,3614,lcl|HF680312.1_prot_CCU73713.1_3264,96.552,116,4,0,1,116,216,331,5.770000e-80,241.0
10554,3614,lcl|HF680312.1_prot_CCU73876.1_3427,96.552,116,4,0,1,116,223,338,8.800000e-80,241.0
10555,3614,lcl|HF680312.1_prot_CCU73197.1_2748,96.552,116,4,0,1,116,223,338,8.800000e-80,241.0


In [15]:
hits = mil1hits[mil1hits['sseqid'].str.contains("CCU")].sort_values('bitscore', ascending=False).drop_duplicates('qseqid')
hits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
2833,1023,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.000000e+00,4296.0
4240,1485,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.000000e+00,4158.0
9142,3217,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.000000e+00,3406.0
2611,947,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.000000e+00,3372.0
9919,3493,lcl|HF680312.1_prot_CCU71621.1_1172,99.690,1611,5,0,1,1611,1,1611,0.000000e+00,3338.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9221,3259,lcl|HF680312.1_prot_CCU73367.1_2918,41.096,73,43,0,5,77,8,80,7.960000e-13,54.7
8055,2816,lcl|HF680312.1_prot_CCU72347.1_1898,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5
10282,3571,lcl|HF680312.1_prot_CCU73193.1_2744,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5
4800,1681,lcl|HF680312.1_prot_CCU72336.1_1887,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5


In [16]:
for i, hit in hits.iterrows():
    gene = genes[str(hit['qseqid'])]
    match = hit['sseqid'].split('_')[2]
    gene.qualifiers['product'] = [mil1genes[match]]

In [19]:
# Бонус

In [ ]:
!wget -nc https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz

In [20]:
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

In [21]:
previd = None
swissgenes = dict()
for line in open('SwissProt_names.txt'):
    if line.startswith('ID'):
        previd = line.split()[1]
    if line.startswith('DE'):
        swissgenes[previd] = line.split('=')[1][:-2]

In [22]:
swisshits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swisshits = swisshits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in swisshits.iterrows():
    gene = genes[str(hit['qseqid'])]
    match = hit['sseqid'].split('|')[-1]
    gene.qualifiers['product'] = [swissgenes[match]]

In [23]:
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

74